In [29]:
csv_filenames = ['syhdr_commercial_inpatient_2016.csv',
 'syhdr_commercial_outpatient_2016.csv',
 'syhdr_commercial_person_2016.csv',
 'syhdr_commercial_pharmacy_2016.csv',
 'syhdr_medicaid_inpatient_2016.csv',
 'syhdr_medicaid_outpatient_2016.csv',
 'syhdr_medicaid_person_2016.csv',
 'syhdr_medicaid_pharmacy_2016.csv',
 'syhdr_medicaid_provider_2016.csv',
 'syhdr_medicare_inpatient_2016.csv',
 'syhdr_medicare_outpatient_2016.csv',
 'syhdr_medicare_person_2016.csv',
 'syhdr_medicare_pharmacy_2016.csv',
 'syhdr_medicare_provider_2016.csv']

In [28]:
import aws_s3_data_handler
csv_paths = aws_s3_data_handler.return_csv_files(csv_filenames)

In [30]:
csv_paths

['s3://processed.lenora.dataset/syhdr_commercial_inpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_commercial_outpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_commercial_person_2016.csv',
 's3://processed.lenora.dataset/syhdr_commercial_pharmacy_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicaid_inpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicaid_outpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicaid_person_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicaid_pharmacy_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicaid_provider_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicare_inpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicare_outpatient_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicare_person_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicare_pharmacy_2016.csv',
 's3://processed.lenora.dataset/syhdr_medicare_provider_2016.csv']

In [ ]:
import data_handler
pandas_df = data_handler.return_pandas_df_from_csv('chosen combobox item')
import cryptography_handler
import os
import pandas as pd
from pandasai import Agent

encrypted_val = b'gAAAAABm9fuEW69QQqx9iJj-d7mprLSD_YGeHMmK4MN91z_CwEAJ6YK9dC2v8_9OXfOVx2My8NV-whI0Nff-OG1iJCJcxh8Mvltm4ATeto4Gl8Rvph9JHbRsMEUHsfaFTmmvLXy0WBhwTIwfslvaytSkzk6kOEWNxjh8t4XZZKY3bebAptDnyNn2leK9tZIdRzTV1iiHym987eA-HmvmPx6RUdWrJHuP3W0g8OAgVG5cUJbth2WeAIM0K5IqBFqarvlvDw7RYVDPnPRy2pwq6VaGrzfTXZDQ4mf6wvQ8NHddIfvQbmPHqpE='
key = cryptography_handler.load_key()
open_ai_key = cryptography_handler.decrypt_api_key(encrypted_val, key)
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import os
os.environ["OPENAI_API_KEY"] = open_ai_key
llm = ChatOpenAI(model = 'gpt-4', temperature = 0)
agent = create_pandas_dataframe_agent(llm, pandas_df,verbose= True, allow_dangerous_code= True)
agent.invoke('What is the most correlated columns, choose 3')

In [ ]:
import data_handler
pandas_df = data_handler.return_pandas_df_from_csv(csv_paths[3])
from pandas_profiling import ProfileReport
profile = ProfileReport(pandas_df, title="commercial_pharmacy", explorative=True)
profile.to_file("commercial_pharmacy.html")